In [11]:
import gurobipy as gp
import pandas as pd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

# Gurobi Environment

When Gurobi is run on local machine, creating an environment will obtain a license, and disposing of the environment will release that license. 

Gurobi environment is created by calling the function `gurobipy.Env()`. The environment is disposed of by calling the function `env.dispose()`.


In [12]:
# Create an environment with your WLS license
params = {
"WLSACCESSID": '915f39ff-e6cb-47c8-913d-e5a46c3209af',
"WLSSECRET": '377acdef-a125-476f-8614-816e5c6c7e80',
"LICENSEID": 2419076,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2419076


Academic license - for non-commercial use only - registered to 2021fc04625@wilp.bits-pilani.ac.in


# Data Sets
Data from ERP and CRM systems was extracted in excel files, cleaned and merged to arrive at two sources of data required for funding allocation to work. **Fund** and **Expense**
Workbook v4 contains two sheets. One for Fund and Another for Expense
## Expense
Expense data has been created by combining data from following sources:
-  **ERP Project Master**
-  **ERP Project Task (Work Breakdown Structure) Details**
-  **ERP Project Cost Details**
-  **Department Hierarchy Master Data**

It shows the values related to Results Based Management (RBM), Earmarking Details and other Strategic Framework dimensions against each expense line.
*Expense Id* has been added additionally for ease of validation of Funding Allocation results.

<div style="font-size: 10px;background-color: #f2f2f2; width: 1200px; height: 200px; overflow: auto;">

|Expense Id	|Project Name	|Task Number	|Expenditure Type	|Budget Year	|Fund	|Budget Category	|Impact Area	|Outcome Area	|Population Group	|SDG	|Budget Situation	|Organizational Marker	|Project Organization	|Level 1	|Region	|Sub region	|Operation	|Country	|Accounting Date	|Amount|
|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------|
|E1	|2023-AFG ABC-ABOD-33021|	|100|	|621110 STF-Salary|	|2023|	|110|	|ABOD|	|0|	|0|	|0|	|0|	|0|	|0|	|Afghanistan, Kabul (33021)|	|Field Operations (PFI)|	|Asia and the Pacific (RASO)|	|South-West Asia (SSWASIA)|	|Afghanistan ABC (OAFG_ABC)|	|Afghanistan|	|44946|	|0|
|E2	|2023-AFG ABC-ABOD-33021	|100	|621110 STF-Salary	|2023	|110	|ABOD	|0	|0	|0	|0	|0	|0	|Afghanistan, Kabul (33021)	|Field Operations (PFI)	|Asia and the Pacific (RASO)	|South-West Asia (SSWASIA)	|Afghanistan ABC (OAFG_ABC)	|Afghanistan	|44946	|0|
</div>

## Fund
Fund data has been created by combining data from following sources:
-  **CRM Contribution Details**
-  **ERP Contract Information**
-  **ERP Contract Project Information**
-  **ERP Revenue Details**

It shows the values related to Results Based Management (RBM), Earmarking Details and other Strategic Framework dimensions against each expense line.
*Fund Id* has been added additionally for ease of validation of Funding Allocation results.

<div style="font-size: 10px;background-color: #f2f2f2; width: 1200px; height: 200px; overflow: auto;">

|Fund Id	|Project Name	|Project Start Date	|Project Finish Date	|Earmarking Level	|Earmarking Type	|Budget Year	|Fund	|Budget Category	|Impact Area	|Outcome Area	|Population Group	|SDG	|Budget Situation	|Organizational Marker	|Organization	|Level 1	|Region	|Sub region	|Operation	|Country	|Output	|Fundraising Situation	|HCR_Carry_Over	|ABOD Amount	|Total Amount	|Amount|
|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------	|------|
|F143	|18-GOV-GB-001-02-125	|43405	|46022	|Tightly Earmarked	|Within_Operation	|2023	|110	|0	|IA1: Protect	|OA1: Access/Doc	|1	|	|	|	|Kenya, Dadaab (12083)	|Field Operations (PFI)	|East Horn and Great Lakes (REHAGL)	|East Horn and Great Lakes (SEHAGL)	|Kenya ABC (OKEN_ABC)	|Kenya	|	|	|Allow Carry Over	|7000	|138064.51	|131064.51|
|F144	|18-GOV-GB-001-02-126	|43405	|46022	|Tightly Earmarked	|Within_Operation	|2023	|110	|0	|IA2: Assist	|OA4: GBV	|1	|	|	|	|Kenya, Dadaab (12083)	|Field Operations (PFI)	|East Horn and Great Lakes (REHAGL)	|East Horn and Great Lakes (SEHAGL)	|Kenya ABC (OKEN_ABC)	|Kenya	|	|	|Allow Carry Over	|3500	|69032.26	|65532.26|

</div>



In [13]:
# Read data
m_expense_data = pd.read_excel("Workbook v4.xlsx", sheet_name='Expense', engine='openpyxl')
m_fund_data = pd.read_excel("Workbook v4.xlsx", sheet_name='Fund', engine='openpyxl')

#Gurobi Model Initialization
Forinitializing a new model of Gurobi of the name "funding_allocation" and environment parameter of name "env".

A model object in Gurobi has some standard methods that we will use in the code. 
-  **model.optimize()** - This method will solve the model and return the status of the model.
-  **model.addVars()** - This method will add multiple variables to the model.
-  **model.addConstrs()** - This method will add multiple constraints to the model.

In [14]:
# Create a Gurobi model
model = gp.Model("funding_allocation", env=env)

In [15]:
pd.options.display.float_format = '{:,.2f}'.format

In [16]:
print("Total 2023 Expense:",f"{m_expense_data[m_expense_data['Budget Year'] == 2023]['Amount'].sum():,.2f}")
print("\n")
print(m_expense_data[m_expense_data['Budget Year'] == 2023].groupby(by='Budget Category')['Amount'].sum())
print("\n\n")
print("Total 2023 Fund:", f"{m_fund_data[m_fund_data['Budget Year'] == 2023]['Amount'].sum():,.2f}")
print("\n")
print(m_fund_data[m_fund_data['Budget Year'] == 2023].groupby(by='Earmarking Level')['Amount'].sum())
print("\n")

Total 2023 Expense: 944,631,722.71


Budget Category
ABOD    109,770,309.14
JPO       1,822,892.09
OPS     401,779,773.47
STAFF   431,258,748.01
Name: Amount, dtype: float64



Total 2023 Fund: 1,225,949,878.35


Earmarking Level
Earmarked           357,811,840.82
Softly Earmarked    154,105,759.38
Tightly Earmarked   370,020,971.00
Unearmarked         344,011,307.15
Name: Amount, dtype: float64




# Data Preparation

## Selecting Data
Following filters are applied to the source data:
-  **Year** For funding allocation, only the expense and fund data assigned to the year 2023 has been selected. 
-  **Non Negative** For this excericse, only the expense and fund data with non-negative amount has been selected.

## Adding Overhead Fund Lines

For each fund line, there may be an assigned value of fund that is charged to donor for administrative purposes. Original contribution is recorded with total amount and the amount that can be spent for administrative purposes. 
We add new fund rows to the source fund data where the fund amount is equal to the overhead amount and earmarking level is set to unearmarked. Setting earmarking level to unearmarked will help in getting using the corresponding fund lines for ABOD and Staff budget category cost.

In [17]:
#select data only from current year and greater than zero
n_fund_data = m_fund_data[(m_fund_data['Amount'] > 0) & (m_fund_data['Budget Year'] == 2023)]
n_expense_data = m_expense_data[(m_expense_data['Amount'] > 0) & (m_expense_data['Budget Year'] == 2023)]

print(n_fund_data.shape)
print(n_expense_data.shape)

# create unearmarked values from the funds based on ABOD amount
# 1: Create a copy of the fund_data
new_rows = n_fund_data.copy()

# 2: Modify the copied data based on the conditions so that new unearmarked rows are created for the overhead amounts
new_rows['Earmarking Level'] = 'Unearmarked'
new_rows['Earmarking Type'] = 'Unearmarked'
new_rows['Fund Id'] = new_rows['Fund Id'] + '_1'
new_rows['Impact Area'] = None
new_rows['Outcome Area'] = None
new_rows['Population Group'] = None
new_rows['SDG'] = None
new_rows['Budget Situation'] = None
new_rows['Organizational Marker'] = None
new_rows['Organization'] = None
new_rows['Level 1'] = 'Headquarters and Global Programmes (PHQGLO)'
new_rows['Region'] = None
new_rows['Sub region'] = None
new_rows['Operation'] = None
new_rows['Country'] = None
new_rows['Output'] = None
new_rows['Fundraising Situation'] = None
new_rows['Total Amount'] = new_rows['ABOD Amount']
new_rows['Amount'] = new_rows['ABOD Amount']
new_rows['ABOD Amount'] = 0

# 3: Concatenate the original fund data and the new rows data
extended_fund_data = pd.concat([n_fund_data, new_rows], axis=0).reset_index(drop=True)

print(extended_fund_data.shape)

print("Total in n_Fund:", f"{n_fund_data['Total Amount'].sum():,.2f}")
print("\n")
print(n_fund_data.groupby(by='Earmarking Level')['Amount'].sum())
print("\n")
print("Total in extended Fund:", f"{extended_fund_data['Amount'].sum():,.2f}")
print("\n")
print(extended_fund_data.groupby(by='Earmarking Level')['Amount'].sum())
print("\n")

(2386, 27)
(347587, 21)
(4772, 27)
Total in n_Fund: 1,543,412,988.72


Earmarking Level
Earmarked           435,162,078.53
Softly Earmarked    217,955,697.22
Tightly Earmarked   474,180,796.32
Unearmarked         344,011,307.15
Name: Amount, dtype: float64


Total in extended Fund: 1,543,412,988.72


Earmarking Level
Earmarked           435,162,078.53
Softly Earmarked    217,955,697.22
Tightly Earmarked   474,180,796.32
Unearmarked         416,114,416.65
Name: Amount, dtype: float64




# Sample Data Creation

## Data Volume
The following data preparation steps has been performed:
-  Filter for one year
-  Removing negative amounts
-  Adding overhead fund lines

Post these steps the data volume is as follows:
-  Fund Data 4772 rows and 27 columns
-  Expense Data  347587 rows and  21 columns

Gurobi creates the variable and constraints data based on following logic
1.   Number of columns: number of rows in Fund * number of rows in expense
2.   Number of rows: sum of number of constraints = Fund constraints from General constraints + Expense rows from general constraints

This creates very large matrix that cannot be solved using the local machine or platforms such as Google Colab pro+. 

## Sample Data
The sample data is created using following logic.
-  A percentage is entered for the total amount of fund and expense data to be used for the sample data.
-  The sample data is created by randomly selecting the rows from the source data based on the percentage entered.
-  For fund data the percentage of amounts by earmarking level is kept same as that of source data
-  For expsne data the percentage of amounts by budget category is kept same as the of source data



In [18]:
def optimized_random_sample_rows(group):
    desired_sum = external_desired_sums[group.name]

    # Early exit if group's total amount is below desired sum
    if group['Amount'].sum() <= desired_sum:
        return group

    indices_chosen = set()
    cumulative_sum = 0
    available_indices = set(group.index)

    while cumulative_sum < desired_sum and available_indices:
        batch_size = max(1, int(0.05 * len(available_indices)))  # Sample 5% of available indices
        sampled_indices = np.random.choice(list(available_indices), size=batch_size, replace=False)

        for idx in sampled_indices:
            cumulative_sum += group.loc[idx, 'Amount']
            indices_chosen.add(idx)
            available_indices.remove(idx)  # Efficiently remove index from set

            if cumulative_sum >= desired_sum:
                break

    return group.loc[list(indices_chosen)]

fund_sample_fraction = .05
external_desired_sums = n_fund_data.groupby('Earmarking Level')['Amount'].sum() * fund_sample_fraction
fund_data = extended_fund_data.groupby('Earmarking Level').apply(optimized_random_sample_rows).reset_index(drop=True)
print(fund_data.shape, f"{fund_data['Amount'].sum():,.2f}")

expense_sample_fraction = .05
external_desired_sums = n_expense_data.groupby('Budget Category')['Amount'].sum() * expense_sample_fraction
expense_data = n_expense_data.groupby('Budget Category').apply(optimized_random_sample_rows).reset_index(drop=True)
print(expense_data.shape, f"{expense_data['Amount'].sum():,.2f}")

print("\n")
print(fund_data.groupby(by='Earmarking Level')['Amount'].sum())
print("\n")
print(expense_data.groupby(by='Budget Category')['Amount'].sum())
print("\n")


(369, 27) 78,890,898.89
(16763, 21) 128,626,525.77


Earmarking Level
Earmarked           23,633,947.16
Softly Earmarked    11,954,786.63
Tightly Earmarked   25,920,888.36
Unearmarked         17,381,276.74
Name: Amount, dtype: float64


Budget Category
ABOD    28,436,437.26
JPO        105,602.51
OPS     75,184,231.81
STAFF   24,900,254.19
Name: Amount, dtype: float64




# Decision Variables
Decision variables capture the result of the optmization. In a feasible solution, the computed values for the decision variables satisfy all of the model constraints. Some of these constraints are associated with individual variables (e.g., variable bounds), while others capture relationships between variables.

## Funding Allocation Variables

The decision variables are formally defined as:

$$ x(i,j) \quad \text{for all } i \in \text{Funds} \text{ and } j \in \text{Expenses} $$

The decision variables for the optimization model represent the allocation of funds from a specific source \( i \) to a particular expense \( j \). These variables are indexed by pairs \((i, j)\), where \( i \) ranges over all fund sources and \( j \) ranges over all expenses.

**decision_var_indices** is a list of tuples where each tuple contains two integers:
-  i: index from fund data
-  j: index from expense data
This variable can take any continuos value between its lower and upper bound. 

**min_amounts** is a numpy array that captures the minimum value from each combination of fund and expense data. It's shape is fund row count * expense row count

**ub_values** ** is a numpy array that stores minimum amounts in the form of a 1D array.

In [19]:
# Convert the Amount columns to numpy arrays for faster access
fund_amounts = fund_data["Amount"].values
expense_amounts = expense_data["Amount"].values

# Precompute the minimum amounts using numpy's outer function, which applies a function on all pairs of values from two arrays
min_amounts = np.minimum.outer(fund_amounts, expense_amounts)

# Calculate the total number of decision variables needed
total_vars = len(fund_data) * len(expense_data)
print(f"Total number of decision variables: {total_vars}")

# Create a flat list of variable names
decision_var_indices = [(i, j) for i in range(len(fund_data)) for j in range(len(expense_data))]

# Flatten the min_amounts matrix for direct usage
ub_values = min_amounts.ravel()



Total number of decision variables: 6185547


# Adding Variables to model
addVars stores the decision variables in the form of tupledict
-  **tupledict:** In particular, a tupledict is a Python dict where the keys are stored as a Gurobi tuplelist, and where the values are typically Gurobi Var objects.
## Parameters of addVars
-  **decision_var_indices:** decision_var_indices provides indices that is used as keys to access the variables. 
-  **VType:** Variable type ('C' for continuous, 'B' for binary, 'I' for integer, 'S' for semi-continuous, or 'N' for semi-integer). 
-  **lb:** Lower Bound is zero. This is an assumption made in this model that it does not cover negative transfer of fund to expense.
-  **ub:** Upper bound is based on ub_values which was calculated in previous cell

In [20]:
x = model.addVars(decision_var_indices, vtype=gp.GRB.CONTINUOUS, lb=0, ub=ub_values, name="x")

#### Pandas to numpy conversion
Converting the columns used for writing constraints as numpy arrays instead of trying to access them as pandas column using iloc.

In [21]:
# Convert each column of expense_data to a numpy array
expense_budget_year = expense_data["Budget Year"].values
expense_fund = expense_data["Fund"].values
expense_budget_category = expense_data["Budget Category"].values
expense_impact_area = expense_data["Impact Area"].values
expense_outcome_area = expense_data["Outcome Area"].values
expense_population_group = expense_data["Population Group"].values
expense_sdg = expense_data["SDG"].values
expense_budget_situation = expense_data["Budget Situation"].values
expense_organizational_marker = expense_data["Organizational Marker"].values
expense_project_organization = expense_data["Project Organization"].values
expense_level_1 = expense_data["Level 1"].values
expense_region = expense_data["Region"].values
expense_sub_region = expense_data["Sub region"].values
expense_operation = expense_data["Operation"].values
expense_country = expense_data["Country"].values

In [22]:
# Convert each column of fund_data to a numpy array
fund_earmarking_level = fund_data["Earmarking Level"].values
fund_earmarking_type = fund_data["Earmarking Type"].values
fund_budget_year = fund_data["Budget Year"].values
fund_fund = fund_data["Fund"].values
fund_budget_category = fund_data["Budget Category"].values
fund_impact_area = fund_data["Impact Area"].values
fund_outcome_area = fund_data["Outcome Area"].values
fund_population_group = fund_data["Population Group"].values
fund_sdg = fund_data["SDG"].values
fund_budget_situation = fund_data["Budget Situation"].values
fund_organizational_marker = fund_data["Organizational Marker"].values
fund_level_1 = fund_data["Level 1"].values
fund_region = fund_data["Region"].values
fund_sub_region = fund_data["Sub region"].values
fund_operation = fund_data["Operation"].values
fund_country = fund_data["Country"].values
fund_output = fund_data["Output"].values
fund_fundraising_situation = fund_data["Fundraising Situation"].values
fund_hcr_carry_over = fund_data["HCR_Carry_Over"].values
fund_abod_amount = fund_data["ABOD Amount"].values


## Priority
Setting priority of the funding allocation so that Tightly earmarked has highest priority and Unearmard as lowest prioruty

In [23]:
priority = {
    "Tightly Earmarked": 4,
    "Earmarked": 3,
    "Softly Earmarked": 2,
    "Unearmarked": 1
}


# Objective Function

The optimization model aims to maximize a weighted sum of allocations, where each allocation corresponds to directing funds from a specific source to a particular expense. The weights are determined based on the earmarking level of the funds. A dictionary, `priority`, provides these weights for each unique earmarking level.

The objective function is given by:

$$\text{Objective} = \sum_i \sum_j \text{weight}[i] \times x[i,j]$$

Where:
- \( x[i, j] \) represents the allocation from the \(i\)-th fund to the \(j\)-th expense.
- The `weight_vector` provides the weight for each fund, and it's constructed using the `priority` dictionary and the `fund_earmarking_level` array.

The model seeks to maximize this objective, ensuring that funds are allocated in a manner that prioritizes certain earmarking levels over others based on the given weights.



In [24]:
  # 1. Vectorized weight calculation
  weights = np.array([priority[key] for key in np.unique(fund_earmarking_level)])
  weight_vector = weights[np.searchsorted(np.unique(fund_earmarking_level), fund_earmarking_level)]

  # Create an objective expression using the precomputed weight_vector and string names as indices
  obj_expr = gp.quicksum(weight_vector[i] * x[i, j] for i in range(len(fund_data)) for j in range(len(expense_data)))

  # Set the objective
  model.setObjective(obj_expr, gp.GRB.MAXIMIZE)



# Constraints
There are three types of constraints that we have built into this model.
-  General Constraints
-  Earmarking Constraints
-  Overhead Cost to Fund Matching Constraints
-  Matching amount constraints

### Constraints Defined in Functions:

#### 1. `General Constraints`:

-  **Description**: 
    - This constraint ensures that the budget year of a fund source \(i\) matches the budget year of an expense \(j\), and the fund type of the fund source \(i\) matches the fund type of the expense \(j\).
- **Mathematical Representation**:
    $$
    \text{if } \text{fund\_budget\_year}[i] \neq \text{expense\_budget\_year}[j] \text{ or fund\_fund}[i] \neq \text{expense\_fund}[j] \text{, then } x(i,j) = 0
    $$
- **Gurobi Constraint Type**: 
    - This is a **Linear Constraint** (`addLConstr`). The decision variable \( x(i,j) \) is set to 0 based on certain conditions.

#### 2. `Overhead Constraints`:
- **Description**: 
    - This constraint ensures that certain expense categories (like 'ABOD' and 'STAFF') are only funded using unearmarked funds or funds explicitly allocated for those purposes.
- **Mathematical Representation**:
    $$
    \text{if } \text{fund\_earmarking\_level}[i] \neq \text{'Unearmarked'} \text{ and expense\_budget\_category}[j] \in \{\text{'ABOD'}, \text{'STAFF'}\} \text{, then } x(i,j) = 0
    $$
- **Gurobi Constraint Type**: 
    - This is a **Linear Constraint** (`addLConstr`). The decision variable \( x(i,j) \) is set to 0 based on certain conditions.

#### 3. `Earmarking Constraints`:
- **Description**: 
    - This constraint is more complex and involves multiple conditions based on the earmarking level of the fund. It ensures that funds are allocated to expenses based on specific earmarking criteria. The criteria can vary based on the earmarking level (Tightly Earmarked, Earmarked, Softly Earmarked) and can involve checks on multiple attributes like operation, country, region, impact area, etc.
- **Mathematical Representation**:
    - This function contains multiple conditional checks and returns a boolean value. If the function returns `False`, the corresponding decision variable \( x(i,j) \) is set to 0.
- **Gurobi Constraint Type**: 
    - This is a **Linear Constraint** (`addLConstr`). The decision variable \( x(i,j) \) is set to 0 based on a comprehensive set of conditions.

### 4. `Total Fund Utilization Constraint`:
- **Description**: 
    - For each fund source \( i \), the sum of allocations to all expenses should not exceed the original amount of that fund. This ensures that funds are not over-allocated.
- **Mathematical Representation**:
    - For each fund \( i \):
    $$
    \sum_{j} x(i,j) \leq \text{Amount of Fund } i
    $$
- **Significance**:
    - This constraint ensures that we do not allocate more from a fund than what is available. It guarantees that the solution remains feasible in terms of fund availability.

### 5. `Total Expense Matched Constraint (Linear Constraint)`
- **Description**: 
    - For each expense \( j \), the sum of allocations from all fund sources should not exceed the amount of that expense. This ensures that expenses are not over-funded.
- **Mathematical Representation**:
    - For each expense \( j \):
    $$
    \sum_{i} x(i,j) \leq \text{Amount of Expense } j
    $$
- **Significance**:
    - This constraint ensures that an expense does not receive more funds than its actual amount. It ensures that the solution remains feasible in terms of expense requirements.


In [25]:
# General Constraints
def general_constraints(i, j):
    if fund_budget_year[i] != expense_budget_year[j] or  fund_fund[i] != expense_fund[j]:
        return False
    else:
        return True


In [26]:
# Overhead Constraint so that STAFF and ABOD categories are met with unearmarked funds only or with funds allocated for that purpose

def overhead_constraints(i, j):
  if fund_earmarking_level[i] != 'Unearmarked' and expense_budget_category[j] in ['ABOD', 'STAFF']:
    return True
  else:
    return False

In [27]:
def xisnan(value):
    return value != value

def earmarking_constraints(i, j):
    if fund_earmarking_level[i] == "Tightly Earmarked":
        fields_to_match = [
            (fund_operation, expense_operation),
            (fund_country, expense_country),
            (fund_impact_area, expense_impact_area),
            (fund_outcome_area, expense_outcome_area)
        ]

        for fund_value, expense_value in fields_to_match:
            if not xisnan(fund_value[i]) and fund_value[i] != expense_value[j]:
                return False
        return True

    elif fund_earmarking_level[i] == "Earmarked":
        if fund_earmarking_type[i] == "Operation":
            return fund_country[i] == expense_country[j]
        else:
            return True

    elif fund_earmarking_level[i] == "Softly Earmarked":
        conditions_map = {
            "Regional_Sub_regional": (
                [fund_region[i], fund_sub_region[i]],
                [expense_region[j], expense_sub_region[j]]
            ),
            "Budget_Situation": ([fund_budget_situation[i]], [expense_budget_situation[j]]),
            "Operation": ([fund_operation[i]], [expense_operation[j]]),
            "Fund Raising Situation AB": ([fund_fundraising_situation[i]], [expense_budget_situation[j]]),
            "Zakat": ([fund_outcome_area[i]], [expense_outcome_area[j]]),
            "Thematic": (
                [
                    fund_fundraising_situation[i],
                    fund_impact_area[i],
                    fund_organizational_marker[i],
                    fund_outcome_area[i]
                ],
                [
                    expense_budget_situation[j],
                    expense_impact_area[j],
                    expense_organizational_marker[j],
                    expense_outcome_area[j]
                ]
            )
        }

        earmarking_type = fund_earmarking_type[i]
        if earmarking_type in conditions_map:
            fund_values, expense_values = conditions_map[earmarking_type]
            for f_val, e_val in zip(fund_values, expense_values):
                if not xisnan(f_val) and f_val != e_val:
                    return False
            return True
    return False



In [28]:
# Step 1: Modify check_constraints
def check_constraints(indices):
    i, j = indices
    try:
        if not (general_constraints(i, j) and earmarking_constraints(i, j) and overhead_constraints(i, j)):
            return (i, j)
    except TypeError:
        # Print out the problematic values
        print(f"Error for indices i={i}, j={j}")
        raise  # Raise the error again after printing
    return None


# Step 2: Filter without parallelization
zero_constraints_indices = [idx for idx in decision_var_indices if check_constraints(idx)]

# Step 3: Add Constraints to the Model
for i, j in zero_constraints_indices:
    x[i, j].UB = 0


In [29]:
# Constraint 1: Total fund utilized should not be more than fund original amount
for i in range(len(fund_data)):
    lhs = gp.LinExpr()  # Create a linear expression for the left-hand side
    lhs.addTerms([1.0] * len(expense_data), [x[i, j] for j in range(len(expense_data))])
    model.addLConstr(lhs <= fund_data.loc[i, "Amount"])

# Constraint 2: Total expense matched should not be more than expense amount
for j in range(len(expense_data)):
    lhs = gp.LinExpr()  # Create a linear expression for the left-hand side
    lhs.addTerms([1.0] * len(fund_data), [x[i, j] for i in range(len(fund_data))])
    model.addLConstr(lhs <= expense_data.loc[j, "Amount"])


In [30]:
# Define the optimization progress callback function
iter_data = []

def optimization_progress(model, where):
    if where == gp.GRB.Callback.MIP:
        objbst = model.cbGet(gp.GRB.Callback.MIP_OBJBST)
        objbnd = model.cbGet(gp.GRB.Callback.MIP_OBJBND)
        runtime = model.cbGet(gp.GRB.Callback.RUNTIME)
        nodes = int(model.cbGet(gp.GRB.Callback.MIP_NODCNT))
        solcnt = int(model.cbGet(gp.GRB.Callback.MIP_SOLCNT))
        iter_data.append((runtime, nodes, solcnt, objbst, objbnd))


In [31]:
model.update()

# Optimize and display results
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to 2021fc04625@wilp.bits-pilani.ac.in
Optimize a model with 17132 rows, 6185547 columns and 12371094 nonzeros
Model fingerprint: 0x6fefd070
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e-02, 1e+06]
  RHS range        [1e-02, 2e+07]
Presolve removed 4215 rows and 5939592 columns
Presolve time: 2.48s
Presolved: 12917 rows, 245955 columns, 490635 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.926e+05
 Factor NZ  : 2.182e+05 (roughly 80 MB of memory)
 Factor Ops : 5.318e+06 (less than 1 second per iteration)
 Threads    : 6

Barrier performed 0 iterations in 2.98 seconds (3.03 work units

# Rows and columns in model


1.   **Number of columns:** number of rows in Fund * number of rows in expense
2.   **Number of rows:** sum of number of constraints = Fund constraints from General constraints + Expense rows from general constraints

# Coefficient Statistics


1.   **Matrix range:** Refers to the range of coefficients in the constraint matrix A of the LP/MIP.
[1e+00, 1e+00] means that all the non-zero coefficients in the constraint matrix are equal to 1.0. This is consistent with this model where we are adding allocations and ensuring they don't exceed certain values. The coefficients for decision variables in these constraints are 1.
2.   **Objective range:** Refers to the range of coefficients in the objective function. [1e+00, 4e+00] indicates that the smallest and largest coefficients in the objective function are 1.0 and 4.0, respectively.
3. **Bounds range:** Refers to the range of variable bounds in the LP/MIP. These are the lower and upper bounds on the decision variables.
[1e-02, 7e+06] means that the smallest bound on any variable is 1 and the largest bound is 7,000,000.
4. **RHS range:** Refers to the range of values on the right-hand side (RHS) of the constraints.
[1e-02, 2e+08] indicates that the smallest RHS value in your constraints is 0.01 and the largest is 200,000,000.


# Concurrent LP Optimizer
Gurobi tries multiple optimizers at the same time. For this scenario it has used:


1.   Dual Simplex is a variation of simplex where...
2.   Barrier method: Instead of moving along the edges of the feasible region (as the simplex methods do), the barrier method moves through the interior of the feasible region.

# Barrier Statistics


1.   **AA' NZ:** This refers to the number of non-zero (NZ) entries in the matrix product AA', where A is the constraint matrix of the LP. The matrix AA' is a key component in the barrier method. Its sparsity (i.e., the number of non-zero entries) can affect the computational efficiency of the method. AA' is 1.307*10^6 non-zero entries.
2.   **Factor NZ:**This refers to the number of non-zero entries in the factorization of the matrix AA'.Factorizing this matrix is a critical step in each iteration of the barrier method. The value indicates that there are 1.762*10^6 on-zero entries in the factorization, and this factorization requires roughly 600 MB of memory.
3. **Factor Ops:** This represents the number of operations (Ops) required to factorize the matrix AA'. The efficiency of the factorization step can significantly affect the overall efficiency of the barrier method. In your output, it takes
9.153*10^7 operations to factorize the matrix, which is estimated to be less than 1 second per iteration.
4. **Threads:** This indicates the number of computational threads used by Gurobi during the barrier method. Using multiple threads allows Gurobi to perform certain calculations in parallel, potentially speeding up the solution process. In your case, Gurobi is using 3 threads.

# Iteration Log
Following table is an example of iteration log displayed.

|          |Objective |          |   Residual|          |      |       |
|:--------:|:--------:|:---------:|:--------:|--------:|:----:|------:|
| Iter     | Primal   | Dual     | Primal    | Dual   | Compl | Time |
|  0.      |  4.99797814e+12    |  9.61411444e+08 |3.76e+09 |2.16e+01|2.81e+06|26s|
|  16  |  9.89618083e+08  |9.89618083e+08|2.85e-07 | 1.07e-14 | 4.14e-10 | 31s |





1.   **Iter:** Column displays iteration number.
2.   **Primal Objective:** Current value of primal objective function.
3. **Dual Objective:** Current value of dual objective function.
4. **Primal Residual:**  It shows primal feasibility residual. It measures how close the current solution is to satisfying all the constraints of the primal problem. A value of zero would mean that all constraints are satisfied.
5. **Dual Residual:** Refers to the dual feasibility residual. It measures how close the current dual solution is to satisfying all the constraints of the dual problem.
6. **Compl:**  This column represents the duality gap, also known as the complementarity gap. It's a measure of the difference between the primal and dual objective values. A smaller duality gap indicates that the solution is closer to optimality. The algorithm aims to reduce this gap with each iteration.

# Crossover

## Barrier Method
Barrier Method and its Output:
The barrier method, as we've discussed, is an interior-point method. It finds a solution that typically lies in the "interior" of the feasible region, rather than at a vertex (corner point) of the feasible region. While this is mathematically valid and gives the correct objective value, many applications and users expect solutions that are basic feasible solutions (BFS) — that is, solutions at the vertices of the feasible region. This is especially true for linear programming problems.

## Summary

To convert the interior solution found by the barrier method to a vertex solution, Gurobi uses a process called "crossover." The crossover step takes the non-vertex solution from the barrier method and iteratively refines it until it reaches a vertex of the feasible region, ensuring that the result is a basic feasible solution.



In [32]:
# Extract values of decision variables and store in a matrix
alloc_matrix = np.array([[x[i, j].x for j in range(len(expense_data))] for i in range(len(fund_data))])

# Convert the matrix to a DataFrame
alloc_df = pd.DataFrame(alloc_matrix, columns=expense_data['Expense Id'], index=fund_data['Fund Id'])

# Melt the DataFrame to get a long format
alloc_df = alloc_df.reset_index().melt(id_vars='Fund Id', value_name='Amount_Allocated').query('Amount_Allocated > 0')

# Calculate the total expense covered and remaining unfunded expense
expense_covered = alloc_df.groupby('Expense Id')['Amount_Allocated'].sum()
expense_data['Covered'] = expense_data['Expense Id'].map(expense_covered).fillna(0)
expense_data['Unfunded'] = expense_data['Amount'] - expense_data['Covered']

# Calculate the fund balance
fund_used = alloc_df.groupby('Fund Id')['Amount_Allocated'].sum()
fund_data['Used'] = fund_data['Fund Id'].map(fund_used).fillna(0)
fund_data['Balance'] = fund_data['Amount'] - fund_data['Used']

# Save to Excel with multiple sheets
with pd.ExcelWriter('output.xlsx') as writer:
    fund_data.to_excel(writer, sheet_name='Fund Data', index=False)
    expense_data.to_excel(writer, sheet_name='Expense Data', index=False)
    alloc_df.to_excel(writer, sheet_name='Allocation Details', index=False)


In [33]:
from google.colab import files
files.download('output.xlsx')

ModuleNotFoundError: No module named 'google.colab'